In [1]:
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss




In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

# 1. 문제 정의

# 2. 데이터 수집

## (1) 데이콘 기본 데이터

In [6]:
os.getcwd()
os.

'C:\\Users\\hongb\\github\\github-repo\\k_nn'

In [9]:
train = pd.read_csv('https://raw.githubusercontent.com/AIdward/github-repo/master/data/train.csv?token=AS2NPXA6XIYXLB5ECHNPS4LAUPQBI').drop(['index'], axis=1).fillna('NAN')
test = pd.read_csv('https://raw.githubusercontent.com/AIdward/github-repo/master/data/test.csv?token=ANBBS4ECWWIBBY4LS522QN3AUPQHE').drop(['index'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('https://raw.githubusercontent.com/AIdward/github-repo/master/data/sample_submission.csv?token=ANBBS4H6UKVZ4L46H3SKPMDAUPQIY')

# 3. 탐색적 데이터 분석

# 4. 변수 조정

In [11]:
# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH
merge_data['1new_1'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['2new_1'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED
merge_data['10new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['11new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED
merge_data['12new_1'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['13new_1'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['14new_1'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# 총 수익을 가족 수로 나누기
merge_data['15new_1'] = merge_data['income_total'] / merge_data['family_size']

# 융합 삭제
#merge_data['3new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['4new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_week']
#merge_data['5new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_month']
#merge_data['6new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_week']

#merge_data['7new_1'] =  merge_data['begin_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['8new_1'] =  merge_data['begin_month'] / merge_data['DAYS_EMPLOYED_month']
#merge_data['9new_1'] =  merge_data['begin_month'] / merge_data['before_EMPLOYED_month']

merge_data['new_1'] = merge_data['child_num'] / merge_data['income_total']
merge_data['new_2'] = merge_data['family_size'] / merge_data['income_total']
merge_data['new_3'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['new_4'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
#merge_data['new_5'] = merge_data['begin_month'] / merge_data['income_total']
merge_data['new_6'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# 소득 skewed-data 처리
merge_data['log1p_income_total'] = np.log1p(merge_data['income_total'])
#merge_data['log_income_total'] = np.log(merge_data['income_total'])
#merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
#merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

In [12]:
from sklearn.neighbors import KNeighborsClassifier

## k-nn

In [14]:
classifier = KNeighborsClassifier(n_neighbors=4)

In [ ]:

training_labels = [0, 1, 1]
classifier.fit(training_points, training_labels)


train = train[train['child_num']<=6].reset_index(drop=True) # 아이의 수가 7명 이상인 데이터 제거

## 인코딩

In [ ]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()


In [ ]:

object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

## Feature 하나씩 빼면서 성능 체크

변수 하나씩 제거하면서 성능 체크<br>
제거하여 성능이 좋게 나온 것들은 리스트에 따로 저장해두기

In [ ]:
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
remove_features = [1, 3, 4, 8, 11, 32,12,13,14,15,16,17]
train = train_copy.drop(train_copy.columns[remove_features], axis=1)
test = test_copy.drop(test_copy.columns[remove_features], axis=1)

train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
train.head()

In [ ]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

In [ ]:
train_copy

In [ ]:
train_x

In [ ]:
lucky_seed = [0,10]
for i, seed in enumerate(lucky_seed):
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv = np.zeros((train_x.shape[0], 3))
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
        lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
        cv[val_idx] = lgbm.predict_proba(x_val)
    print(f'{i+1} multi_logloss: {log_loss(train_y, cv)}')

## 저장한 변수 지우는 코드

In [ ]:
remove_features = [1, 3, 4, 8, 11, 32]
train = train.drop(train.columns[remove_features], axis=1)
test = test.drop(test.columns[remove_features], axis=1)

## 다시 인코딩

In [ ]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [ ]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [ ]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

# 6. 모델 학습

In [ ]:
pred_dict = {}
pred_test_dict = {}

## (1) Lightgbm

In [ ]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

### Parameter Tuning

In [ ]:
SEED=42

# Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
def score(params):
    print("Training with params: ")
    print(params)
    
    kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        lgbmodel = LGBMClassifier(**params)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=100) 
        cv[val_idx, :] = lgbmodel.predict_proba(x_val)
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
    print('multi_logloss:', log_loss(train_y, cv))
    score = log_loss(train_y, cv)
    return {'loss': score, 'status': STATUS_OK}

# Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
def optimize(random_state=SEED):
    
#     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
#          'eta': 0.01, 'min_child_weight': 3,
#          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
#         }
    space = {
        'learning_rate': hp.quniform('learning_rate', 0.003, 0.006, 0.001),
        #'learning_rate' : 0.005,
        'num_leaves': scope.int(hp.quniform('num_leaves', 1000, 1200, 50)),
        'num_leaves' : 1000,
        #'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),
        'min_child_weight' : 2,
        #'subsample': hp.quniform('subsample', 0.8, 1, 0.05),
        'subsample' : 1,
        'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
        #'colsample_bytree' : 0.4,
        'max_depth' : -1,
        'n_estimators' : 5000,
        'objective' : 'multiclass',
        'num_class' : 3,
        'seed': SEED,
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=12)
    return best

best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

### 3 seeds x 5 folds

In [ ]:
lucky_seeds=[2019,91373] # Lucky seed 늘려가면서 하기

for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True) # CV 늘려가면서 하기
    cv=np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        lgbmodel = LGBMClassifier(learning_rate=0.1, objective='multiclass', n_estimators=10000, num_leaves=1000, 
                                  max_depth=-1, min_child_weight=2, colsample_bytree=0.55,  
                                   n_jobs=-1, random_state=seed)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=100) 
        #joblib.dump(lgbmodel, f'./pred_pkl/LGB_{n+1}_fold_{seed}_seed_lgb.pkl')

        # CROSS-VALIDATION , EVALUATE CV
        cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_x) / 5 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
    pred_dict['lgb'+str(i+1)] = cv
    pred_test_dict['lgb'+str(i+1)] = pred_test
        
    print('multi_logloss :', log_loss(train_y, cv))

lgbmodels_path = os.listdir('./pred_pkl/')
lgbmodels_list = [x for x in lgbmodels_path if x.endswith("lgb.pkl")]
assert len(lgbmodels_list) == 15
lgb_preds = np.zeros((test_x.shape[0], 3))

for m in lgbmodels_list:
    lgbmodel = joblib.load('./pred_pkl/'+m)
    lgb_preds_proba = lgbmodel.predict_proba(test)
    lgb_preds += lgb_preds_proba/15

# 결과 제출

In [ ]:
submission = sample_submission.copy()
submission.iloc[:, 1:] = pred_test

In [ ]:
submission.to_csv('submission/submission.csv', index=False)